<a href="https://colab.research.google.com/github/couplewith/colab/blob/main/%EC%A3%BC%EC%8B%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91%EA%B3%BC_%EB%B6%84%EC%84%9D_AMZN_2408.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 수집과 분석을 자동화하기 위해서는 웹 스크래핑, API 사용, 그리고 자동화된 데이터 분석 도구를 활용할 수 있습니다. 여기서는 각 방법에 대한 상세한 설명과 예제를 제공하겠습니다.

# 주식 데이터 수집과 분석

주식 데이터 수집 : 인터넷 주식 정보 사이트를 통해 주식 정보를 수집하고 분석하여 향후 동향을 분석하는 알고리즘을 만들어 보고자 합니다.


## 1. 데이터 수집

- Yahoo Finance: Yahoo Finance - Amazon
https://finance.yahoo.com/quote/AMZN/history?p=AMZN

- Google Finance: Google Finance - Amazon
https://www.google.com/finance/quote/AMZN:NASDAQ

- Bloomberg: Bloomberg - Amazon
https://www.bloomberg.com/quote/AMZN:US

위의 주식은 다음과 같은 내용을 가지고 있습니다.





In [6]:
 # 파이썬시작
 !pip install requests beautifulsoup4 pandas


## 2.2 데이터수집 -  웹 스크래핑

웹 스크래핑을 통해 웹사이트에서 데이터를 자동으로 수집할 수 있습니다. Python의 BeautifulSoup과 requests 라이브러리를 사용하여 데이터를 수집할 수 있습니다



** 예제: Yahoo Finance에서 데이터 수집 **

In [8]:
#예제: Yahoo Finance에서 데이터 수집 - CSV파일
import pandas as pd

# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(url)

# 데이터프레임 출력
print(df.head())

         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume  
0  42832100  
1  47148700  
2  42781500  
3  41675900  
4  48354100  


In [7]:
#예제: Yahoo Finance에서 데이터 수집 - HTML 파싱
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL 설정
url = 'https://finance.yahoo.com/quote/AMZN/history?p=AMZN'

# 요청 보내기
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 테이블 데이터 찾기
table = soup.find('table', {'data-test': 'historical-prices'})

# 데이터프레임으로 변환
rows = table.find_all('tr')
data = []
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

df = pd.DataFrame(data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

# 데이터 전처리
df = df.dropna()

print(df.head())


AttributeError: 'NoneType' object has no attribute 'find_all'

## 2.2 데이터수집 - API 사용
많은 금융 데이터 제공업체는 API를 통해 데이터를 제공합니다. 예를 들어, Alpha Vantage, Yahoo Finance API, Quandl 등이 있습니다.


** alphavantage API 신청 **
https://www.alphavantage.co/support/#api-key

 Free API 신청으로 하루 25건의 이용이 가능합니다.

 `T5HM0TJESBEDRM94`

** 예제: Alpha Vantage API 사용 **

In [5]:
#예제: Alpha Vantage API 사용
import requests
import pandas as pd

# Alpha Vantage API 키
api_key = 'YOUR_API_KEY'
api_key = 'T5HM0TJESBEDRM94'

# URL 설정
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMZN&apikey={api_key}&outputsize=full&datatype=csv'

# 데이터 요청
response = requests.get(url)
data = response.text

# 데이터프레임으로 변환
df = pd.read_csv(pd.compat.StringIO(data))

# 데이터 전처리
df = df.dropna()

print(df.head())

AttributeError: module 'pandas.compat' has no attribute 'StringIO'

In [11]:
!pip install statsmodels pandas requests

### AIC(Akaike Information Criterion)와 BIC(Bayesian Information Criterion)는 모델의 적합성을 평가하는 데 사용되는 통계적 기준입니다.

이 기준들을 사용하여 최적의 p 값을 찾는 방법을 설명해드릴게요.

### AIC와 BIC의 정의
AIC (Akaike Information Criterion): 모델의 적합도와 복잡성을 고려하여 모델을 평가합니다.

 - AIC 값이 낮을수록 모델이 더 적합하다고 판단합니다.
- BIC (Bayesian Information Criterion): AIC와 유사하지만, 모델의 복잡성에 더 큰 패널티를 부여합니다. BIC 값이 낮을수록 모델이 더 적합하다고 판단합니다.
### 최적의 p 값을 찾는 방법
- 모델 생성 및 학습: 여러 가지 p 값을 사용하여 ARIMA 모델을 생성하고 학습합니다.
- AIC와 BIC 계산: 각 모델의 AIC와 BIC 값을 계산합니다.
- 최적의 모델 선택: AIC와 BIC 값이 가장 낮은 모델을 선택합니다.

In [17]:
# AIC (Akaike Information Criterion) , BIC (Bayesian Information Criterion): 찾는법
#
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(url)

# 데이터 전처리
df['Close'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()

# 최적의 p 값을 찾기 위한 변수 초기화
best_aic = float('inf')
best_bic = float('inf')
best_p = None

# 여러 가지 p 값을 사용하여 모델 학습 및 평가
for p in range(1, 11):
    model = ARIMA(df['Close'], order=(p, 1, 0))
    model_fit = model.fit()

    aic = model_fit.aic
    bic = model_fit.bic

    if aic < best_aic:
        best_aic = aic
        best_p_aic = p

    if bic < best_bic:
        best_bic = bic
        best_p_bic = p

print(f"최적의 p 값 (AIC 기준): {best_p_aic}, AIC: {best_aic}")
print(f"최적의 p 값 (BIC 기준): {best_p_bic}, BIC: {best_bic}")

# 2024.8.11
# 최적의 p 값 (AIC 기준): 3, AIC: -318.0206586487865
# 최적의 p 값 (BIC 기준): 2, BIC: -306.8721183453865

최적의 p 값 (AIC 기준): 3, AIC: -318.0206586487865
최적의 p 값 (BIC 기준): 2, BIC: -306.8721183453865



## 4.수집한 데이터 분석


# 모델 학습
model = ARIMA(df['Close'], order=(5, 1, 0))
model_fit = model.fit()

ARIMA 모델을 사용하여 주가 데이터를 학습하는 과정입니다. 각 부분을 설명해드릴게요:

** ARIMA 모델 생성: **
Python
```
model = ARIMA(df['Close'], order=(5, 1, 0))
```

AI가 생성한 코드입니다. 신중하게 검토하고 사용하세요.
FAQ의 자세한 정보.
ARIMA는 AutoRegressive Integrated Moving Average의 약자로, 시계열 데이터를 예측하는 데 사용되는 통계 모델입니다.

- df['Close']는 종가 데이터를 의미합니다.
- order=(5, 1, 0)는 ARIMA 모델의 파라미터를 설정합니다. 여기서 (p, d, q)로 구성되며, 각각의 의미는 다음과 같습니다:
 * p: AR(자기회귀) 부분의 차수 (여기서는 5)
 * d: 차분 횟수 (여기서는 1)
 * q: MA(이동평균) 부분의 차수 (여기서는 0)


** 모델 학습:**
Python
```
model_fit = model.fit()
```
AI가 생성한 코드입니다.
 FAQ의 자세한 정보.
model.fit()은 주어진 데이터에 대해 ARIMA 모델을 학습시키는 과정입니다.
model_fit은 학습된 모델 객체를 저장합니다

In [31]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df1 = pd.read_csv(url)

# 데이터프레임 출력
print(df1.tail())

# 최근 3개월 데이터를 이용한 결과

df = df1.tail(400)

# 데이터 전처리
df['Close2'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()

# 모델 학습
model = ARIMA(df['Close2'], order=(3, 1, 0))  # 최적의 p 값 (AIC 기준) 사용
model_fit = model.fit()

# 최근 7일 간의 예측
forecast_7d = model_fit.forecast(steps=7)

# 예측값을 실제 값으로 변환
mean_close = df['Close'].mean()
std_close = df['Close'].std()
forecast_7d_actual = forecast_7d * std_close + mean_close

print("> mean_close", mean_close,"  >> std_close:", std_close)
print("최근 7일 간의 예측 주가 (실제 값):", forecast_7d_actual)


# 주가 예측
forecast_7d = model_fit.forecast(steps=7) * std_close + mean_close
forecast_14d = model_fit.forecast(steps=14) * std_close + mean_close
forecast_20d = model_fit.forecast(steps=20) * std_close + mean_close
forecast_3m = model_fit.forecast(steps=90) * std_close + mean_close
forecast_6m = model_fit.forecast(steps=180) * std_close + mean_close


print("7Day 예측 주가:", forecast_7d)
print("14Day 예측 주가:", forecast_14d)
print("20Day 예측 주가:", forecast_20d)

print("3개월 예측 주가:", forecast_3m)
print("6개월 예측 주가:", forecast_6m)



           Date        Open        High         Low       Close   Adj Close  \
246  2024-08-05  154.210007  162.960007  151.610001  161.020004  161.020004   
247  2024-08-06  161.710007  165.080002  158.539993  161.929993  161.929993   
248  2024-08-07  166.550003  167.580002  161.429993  162.770004  162.770004   
249  2024-08-08  165.169998  166.690002  162.550003  165.800003  165.800003   
250  2024-08-09  166.399994  168.550003  165.850006  166.940002  166.940002   

       Volume  
246  83149400  
247  59950800  
248  48408200  
249  44616200  
250  35758700  
> mean_close 162.0841036932271   >> std_close: 21.921806775864976
최근 7일 간의 예측 주가 (실제 값): 251    166.505227
252    166.539777
253    166.747870
254    166.715282
255    166.673141
256    166.697181
257    166.704680
Name: predicted_mean, dtype: float64
7Day 예측 주가: 251    166.505227
252    166.539777
253    166.747870
254    166.715282
255    166.673141
256    166.697181
257    166.704680
Name: predicted_mean, dtype: float64
14

<ipython-input-31-c2630987c97e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Close2'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()


In [35]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df1 = pd.read_csv(url)


df = df1.tail(200).head(100)
# 데이터프레임 출력
print(df.head())

# 데이터 전처리
df.loc[:, 'Close2'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()

# 모델 학습
model = ARIMA(df['Close2'], order=(3, 1, 0))  # 최적의 p 값 (AIC 기준) 사용
model_fit = model.fit()

# 주가 예측
forecast_7d = model_fit.forecast(steps=7)
forecast_14d = model_fit.forecast(steps=14)
forecast_20d = model_fit.forecast(steps=20)
forecast_3m = model_fit.forecast(steps=90)
forecast_6m = model_fit.forecast(steps=180)
forecast_12m = model_fit.forecast(steps=365)

print("7일 예측 주가:", forecast_7d)
print("14일 예측 주가:", forecast_14d)
print("20일 예측 주가:", forecast_20d)
print("3개월 예측 주가:", forecast_3m)
print("6개월 예측 주가:", forecast_6m)
print("12개월 예측 주가:", forecast_12m)

print("예측값은 표준화 값으로 역표준화 값으로 실제 가격 도출-----------")

mean_close = df['Close'].mean()
std_close = df['Close'].std()

forecast_7d_actual = forecast_7d * std_close + mean_close
forecast_14d_actual = forecast_14d * std_close + mean_close
forecast_20d_actual = forecast_20d * std_close + mean_close
forecast_3m_actual = forecast_3m * std_close + mean_close
forecast_6m_actual = forecast_6m * std_close + mean_close
forecast_12m_actual = forecast_12m * std_close + mean_close

print("7일 예측 주가 (실제 값):", forecast_7d_actual)
print("14일 예측 주가 (실제 값):", forecast_14d_actual)
print("20일 예측 주가 (실제 값):", forecast_20d_actual)
print("3개월 예측 주가 (실제 값):", forecast_3m_actual)
print("6개월 예측 주가 (실제 값):", forecast_6m_actual)
print("12개월 예측 주가 (실제 값):", forecast_12m_actual)

          Date        Open        High         Low       Close   Adj Close  \
51  2023-10-24  127.739998  128.800003  126.339996  128.559998  128.559998   
52  2023-10-25  126.040001  126.339996  120.790001  121.389999  121.389999   
53  2023-10-26  120.629997  121.639999  118.349998  119.570000  119.570000   
54  2023-10-27  126.199997  130.020004  125.519997  127.739998  127.739998   
55  2023-10-30  129.720001  133.000000  128.559998  132.710007  132.710007   

       Volume  
51   46477400  
52   74577500  
53  100419500  
54  125309300  
55   72485500  
7일 예측 주가: 151    1.465209
152    1.465391
153    1.437072
154    1.436690
155    1.445076
156    1.445287
157    1.442805
Name: predicted_mean, dtype: float64
14일 예측 주가: 151    1.465209
152    1.465391
153    1.437072
154    1.436690
155    1.445076
156    1.445287
157    1.442805
158    1.442713
159    1.443448
160    1.443483
161    1.443266
162    1.443253
163    1.443317
164    1.443322
Name: predicted_mean, dtype: float64
20일 

In [15]:
# model = ARIMA(df['Close'], order=(5, 1, 0))
#
import requests
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA


# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(url)

close_mean = df['Close'].mean()
close_std  = df['Close'].std()

# 데이터 전처리
df['Close2'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()

# 데이터프레임 출력
print(">>> close : ", close_mean, close_std)
print(df.head())
print(df.tail())


# 모델 학습
model = ARIMA(df['Close2'], order=(5, 1, 0))
model_fit = model.fit()

# 주가 예측
forecast_7d = model_fit.forecast(steps=7)
forecast_14d = model_fit.forecast(steps=14)
forecast_20d = model_fit.forecast(steps=20)
forecast_3m = model_fit.forecast(steps=90)
forecast_6m = model_fit.forecast(steps=180)
forecast_12m = model_fit.forecast(steps=365)

print("7Day 예측 주가:", forecast_7d)
print("14Day 예측 주가:", forecast_14d)
print("20Day 예측 주가:", forecast_20d)

print("3개월 예측 주가:", forecast_3m)
print("6개월 예측 주가:", forecast_6m)
print("12개월 예측 주가:", forecast_12m)

print("예측값은 표준화 값으로 역표준화 값으로 실제 가격 도출-----------")

mean_close = df['Close2'].mean()
std_close = df['Close2'].std()


print(">>> std", std_close)
print(">>> mean", mean_close)

forecast_7d_actual = forecast_7d * std_close + mean_close
forecast_14d_actual = forecast_14d * std_close + mean_close
forecast_20d_actual = forecast_20d * std_close + mean_close

forecast_3m_actual = forecast_3m * std_close + mean_close
forecast_6m_actual = forecast_6m * std_close + mean_close
forecast_12m_actual = forecast_12m * std_close + mean_close


print("7Day 예측 주가(실제값):", forecast_7d_actual)
print("14Day 예측 주가(실제값):", forecast_14d_actual)
print("20Day 예측 주가(실제값):", forecast_20d_actual)

print("3개월 예측 주가 (실제 값):", forecast_3m_actual)
print("6개월 예측 주가 (실제 값):", forecast_6m_actual)
print("12개월 예측 주가 (실제 값):", forecast_12m_actual)

>>> close :  162.0841036932271 21.921806775864976
         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume    Close2  
0  42832100 -1.079934  
1  47148700 -0.981402  
2  42781500 -1.113690  
3  41675900 -1.232293  
4  48354100 -1.282016  
           Date        Open        High         Low       Close   Adj Close  \
246  2024-08-05  154.210007  162.960007  151.610001  161.020004  161.020004   
247  2024-08-06  161.710007  165.080002  158.539993  161.929993  161.929993   
248  2024-08-07  166.550003  167.580002  161.429993  162.770004  162.770004   
249  2024-08-08  165.169998



```
# 코드로 형식 지정됨
         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume  
0  42832100  
1  47148700  
2  42781500  
3  41675900  
4  48354100  
3개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
336    0.217807
337    0.217807
338    0.217807
339    0.217807
340    0.217807
Name: predicted_mean, Length: 90, dtype: float64
6개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
426    0.217807
427    0.217807
428    0.217807
429    0.217807
430    0.217807
Name: predicted_mean, Length: 180, dtype: float64
12개월 예측 주가: 251    0.201493
252    0.204488
253    0.217919
254    0.218669
255    0.216105
         ...   
611    0.217807
612    0.217807
613    0.217807
614    0.217807
615    0.217807
Name: predicted_mean, Length: 365, dtype: float64
```



# 표준 편차를 이용한 데이터 복구

```
data = {'Close': [137.40, 140.57, 137.67, 135.07, 133.98]}
df = pd.DataFrame(data)
df.tail(3)  마지막 3개의 값
df.tail(3).mean() : 마지막 3개의 평균
```



In [16]:
import requests
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(url)

# 데이터프레임 출력
print(df.head())

# 데이터 전처리
df['Close'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()

# 모델 학습
model = ARIMA(df['Close'], order=(3, 1, 0))  # 최적의 p 값 (AIC 기준) 사용
model_fit = model.fit()

# 주가 예측
forecast_7d = model_fit.forecast(steps=7)
forecast_14d = model_fit.forecast(steps=14)
forecast_20d = model_fit.forecast(steps=20)
forecast_3m = model_fit.forecast(steps=90)
forecast_6m = model_fit.forecast(steps=180)
forecast_12m = model_fit.forecast(steps=365)

print("7일 예측 주가:", forecast_7d)
print("14일 예측 주가:", forecast_14d)
print("20일 예측 주가:", forecast_20d)
print("3개월 예측 주가:", forecast_3m)
print("6개월 예측 주가:", forecast_6m)
print("12개월 예측 주가:", forecast_12m)

print("예측값은 표준화 값으로 역표준화 값으로 실제 가격 도출-----------")

mean_close = df['Close'].mean()
std_close = df['Close'].std()

print(">>> std", std_close)
print(">>> mean", mean_close)

forecast_7d_actual = forecast_7d * std_close + mean_close
forecast_14d_actual = forecast_14d * std_close + mean_close
forecast_20d_actual = forecast_20d * std_close + mean_close
forecast_3m_actual = forecast_3m * std_close + mean_close
forecast_6m_actual = forecast_6m * std_close + mean_close
forecast_12m_actual = forecast_12m * std_close + mean_close

print("7일 예측 주가 (실제 값):", forecast_7d_actual)
print("14일 예측 주가 (실제 값):", forecast_14d_actual)
print("20일 예측 주가 (실제 값):", forecast_20d_actual)
print("3개월 예측 주가 (실제 값):", forecast_3m_actual)
print("6개월 예측 주가 (실제 값):", forecast_6m_actual)
print("12개월 예측 주가 (실제 값):", forecast_12m_actual)


         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume  
0  42832100  
1  47148700  
2  42781500  
3  41675900  
4  48354100  
7일 예측 주가: 251    0.201677
252    0.203253
253    0.212746
254    0.211259
255    0.209337
256    0.210433
257    0.210775
Name: predicted_mean, dtype: float64
14일 예측 주가: 251    0.201677
252    0.203253
253    0.212746
254    0.211259
255    0.209337
256    0.210433
257    0.210775
258    0.210386
259    0.210392
260    0.210507
261    0.210477
262    0.210451
263    0.210467
264    0.210470
Name: predicted_mean, dtype: float64
20일 예측 주가: 251    0.20


# 최적의 p 값 찾기

```
최적의 p 값 (AIC 기준): 3, AIC: -318.0206586487865
최적의 p 값 (BIC 기준): 2, BIC: -306.8721183453865

라면

model = ARIMA(df['Close'], order=(5, 1, 0))   는 어떤 것이  가능하나요
```


최적의 p 값을 찾는 과정에서 AIC 기준으로는 p=3, BIC 기준으로는 p=2가 최적의 값으로 나타났습니다. 이 경우, model = ARIMA(df['Close'], order=(5, 1, 0))는 최적의 p 값과는 다소 차이가 있습니다.


** AIC 기준으로 최적의 p 값 사용: **
```
# Python
model = ARIMA(df['Close'], order=(3, 1, 0))
```
AIC 기준으로 최적의 p 값인 3을 사용하여 모델을 생성합니다.
AIC는 모델의 적합도와 복잡성을 고려하므로, 이 값을 사용하는 것이 일반적으로 더 나은 예측 성능을 제공할 수 있습니다.


** BIC 기준으로 최적의 p 값 사용: **

```
#Python
model = ARIMA(df['Close'], order=(2, 1, 0))
```

BIC 기준으로 최적의 p 값인 2를 사용하여 모델을 생성합니다.
BIC는 모델의 복잡성에 더 큰 패널티를 부여하므로, 더 단순한 모델을 선호할 때 유용합니다.




** 기존의 p 값 사용: **
```
#Python
model = ARIMA(df['Close'], order=(5, 1, 0))
```

기존의 p 값인 5를 사용하여 모델을 생성합니다.
이 경우, 모델이 더 많은 과거 데이터를 고려하므로, 데이터의 패턴을 더 잘 포착할 수 있지만 과적합의 위험이 있을 수 있습니다.


In [18]:
# model = ARIMA(df['Close'], order=(3, 1, 0)) 적용
#
import requests
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA


# CSV 파일 URL
url = 'https://query1.finance.yahoo.com/v7/finance/download/AMZN?period1=1691730584&period2=1723352984&interval=1d&events=history&includeAdjustedClose=true'

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(url)

# 데이터프레임 출력
print(df.head())


# 데이터 전처리
df['Close2'] = (df['Close'] - df['Close'].mean()) / df['Close'].std()


# 모델 학습
model = ARIMA(df['Close'], order=(3, 1, 0))
model_fit = model.fit()

# 주가 예측
forecast_7d = model_fit.forecast(steps=7)
forecast_14d = model_fit.forecast(steps=14)
forecast_20d = model_fit.forecast(steps=20)
forecast_3m = model_fit.forecast(steps=90)
forecast_6m = model_fit.forecast(steps=180)
forecast_12m = model_fit.forecast(steps=365)

print("7Day 예측 주가:", forecast_7d)
print("14Day 예측 주가:", forecast_14d)
print("20Day 예측 주가:", forecast_20d)

print("3개월 예측 주가:", forecast_3m)
print("6개월 예측 주가:", forecast_6m)
print("12개월 예측 주가:", forecast_12m)

print("예측값은 표준화 값으로 역표준화 값으로 실제 가격 도출-----------")

mean_close = df['Close'].mean()
std_close = df['Close'].std()

forecast_7d_actual = forecast_7d * std_close + mean_close
forecast_14d_actual = forecast_14d * std_close + mean_close
forecast_20d_actual = forecast_20d * std_close + mean_close

forecast_3m_actual = forecast_3m * std_close + mean_close
forecast_6m_actual = forecast_6m * std_close + mean_close
forecast_12m_actual = forecast_12m * std_close + mean_close


print("7Day 예측 주가(실제값):", forecast_7d_actual)
print("14Day 예측 주가(실제값):", forecast_14d_actual)
print("20Day 예측 주가(실제값):", forecast_20d_actual)

print("3개월 예측 주가 (실제 값):", forecast_3m_actual)
print("6개월 예측 주가 (실제 값):", forecast_6m_actual)
print("12개월 예측 주가 (실제 값):", forecast_12m_actual)

         Date        Open        High         Low       Close   Adj Close  \
0  2023-08-11  137.399994  139.330002  137.000000  138.410004  138.410004   
1  2023-08-14  138.300003  140.589996  137.750000  140.570007  140.570007   
2  2023-08-15  140.050003  141.279999  137.229996  137.669998  137.669998   
3  2023-08-16  137.190002  137.270004  135.009995  135.070007  135.070007   
4  2023-08-17  135.460007  136.089996  133.529999  133.979996  133.979996   

     Volume  
0  42832100  
1  47148700  
2  42781500  
3  41675900  
4  48354100  
7Day 예측 주가: 251    0.201677
252    0.203253
253    0.212746
254    0.211259
255    0.209337
256    0.210433
257    0.210775
Name: predicted_mean, dtype: float64
14Day 예측 주가: 251    0.201677
252    0.203253
253    0.212746
254    0.211259
255    0.209337
256    0.210433
257    0.210775
258    0.210386
259    0.210392
260    0.210507
261    0.210477
262    0.210451
263    0.210467
264    0.210470
Name: predicted_mean, dtype: float64
20Day 예측 주가: 251  



## 데이터 분석 및 예측 결과 해석
데이터프레임
주어진 데이터프레임은 아마존 주가의 일일 데이터를 포함하고 있습니다. 각 열의 의미는 다음과 같습니다:
```
Date: 날짜
Open: 시가
High: 고가
Low: 저가
Close: 종가
Adj Close: 수정 종가
Volume: 거래량

** 예측 결과 **
ARIMA 모델을 사용하여 3개월, 6개월, 12개월 동안의 주가를 예측한 결과는 다음과 같습니다:

- 3개월 예측 주가:
예측된 주가는 약 0.201493에서 0.217807 사이입니다.
이는 표준화된 값으로, 실제 주가로 변환하려면 역표준화를 해야 합니다.


- 6개월 예측 주가:
예측된 주가는 약 0.201493에서 0.217807 사이입니다.
3개월 예측과 유사한 범위로, 주가가 안정적인 상승세를 보일 것으로 예상됩니다.

- 12개월 예측 주가:
예측된 주가는 약 0.201493에서 0.217807 사이입니다.

장기적으로도 주가가 안정적인 상승세를 유지할 것으로 보입니다.
```



## Reference
<pre>
Markdown	Preview
**bold text**	bold text
*italicized text* or _italicized text_	italicized text
`Monospace`	Monospace
~~strikethrough~~	strikethrough
[A link](https://www.google.com)	A link
![An image](https://www.google.com/images/rss.png)	An image
</pre>

[참조 : 마크다운 - 작성가이드](https://colab.research.google.com/notebooks/markdown_guide.ipynb#scrollTo=Lhfnlq1Surtk)